In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from xgboost import XGBClassifier

In [ ]:
df = pd.read_csv('Structural data_coarse-grained.csv')
y = df['AMI'].values
X = df.drop(['AMI'], axis=1).values

In [ ]:
#Validatoin A, B, C
#validation A
selected_features = []
#validation B
#selected_features = [ 1,2,4,5,6,7,9,10,11,12, 17,19,20,21,22,23,24,25,26,27,28,30,31,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50] #验证C
#validation C
#selected_features = [29,51,14,32,13,15,16,18,8,3,0,52] 
total_features=np.arange(54).reshape((1, 54))
counter_features=np.setdiff1d(total_features,selected_features)

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, roc_auc_score, log_loss
from xgboost import XGBClassifier
metrics_per_fold = []
praf = ['precision', 'recall', 'acc', 'f1', 'roc_auc', 'log_loss', 'specificity']
NUM_SPLITS = 10
skf = StratifiedKFold(shuffle=True, random_state=79, n_splits=NUM_SPLITS)
for fold, (train_idx, val_idx) in enumerate(skf.split(X=X, y=y)):
    X_train_modified = X[train_idx].copy()
    X_val_modified = X[val_idx].copy().astype(float)
    X_val_modified[:, selected_features] = np.nan
    xgb = XGBClassifier()
    xgb.fit(X_train_modified, y[train_idx])
    pred_proba = xgb.predict_proba(X_val_modified)
    pred_label = np.round(pred_proba[:, 1])  # 根据阈值0.5生成预测标签
    y_val = y[val_idx]
    precision = precision_score(y_val, pred_label)
    recall = recall_score(y_val, pred_label)
    acc = accuracy_score(y_val, pred_label)
    f1 = f1_score(y_val, pred_label)
    roc_auc = roc_auc_score(y_val, pred_proba[:, 1])
    ll = log_loss(y_val, pred_proba[:, 1])
    specificity = recall_score(y_val, pred_label, pos_label=0)
    metrics = [precision, recall, acc, f1, roc_auc, ll, specificity]
    metrics_per_fold.append(metrics)
    print(f"Fold {fold + 1}:")
    for name, value in zip(praf, metrics):
        print(f"  {name}: {value:.4f}")

metrics_per_fold = np.array(metrics_per_fold)
mean_metrics = np.mean(metrics_per_fold, axis=0)
std_metrics = np.std(metrics_per_fold, axis=0, ddof=1)  # 无偏标准差
print("\nOverall Performance (Mean ± Std):")
for name, mean, std in zip(praf, mean_metrics, std_metrics):
    print(f"{name}: {mean:.4f} ± {std:.4f}")
